# 用keras做鐵達尼號

In [1]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

import pandas as pd
import numpy as np
import talib as ta
import pandas_datareader as data
import pandas_datareader.data as web
import datetime
import math

C:\Users\user\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
submit = pd.read_csv('gender_submission.csv')

In [3]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [5]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [6]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [7]:
test.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


# 從上面選取自己要的特徵

In [8]:
selected_features = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Sex', 'Embarked']

In [9]:
X_train = train[selected_features]

In [10]:
y_train = train['Survived']

In [11]:
X_test = test[selected_features]

# 處理缺失值

In [12]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
Pclass      891 non-null int64
Age         714 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
Sex         891 non-null object
Embarked    889 non-null object
dtypes: float64(2), int64(3), object(2)
memory usage: 48.8+ KB


In [13]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 7 columns):
Pclass      418 non-null int64
Age         332 non-null float64
SibSp       418 non-null int64
Parch       418 non-null int64
Fare        417 non-null float64
Sex         418 non-null object
Embarked    418 non-null object
dtypes: float64(2), int64(3), object(2)
memory usage: 22.9+ KB


In [14]:
X_train['Age'].fillna(X_train['Age'].mean(), inplace=True)
X_train['Embarked'].fillna('S', inplace=True)
X_test['Age'].fillna(X_test['Age'].mean(), inplace=True)
X_test['Fare'].fillna(X_test['Fare'].mean(), inplace=True)

C:\Users\user\Anaconda3\lib\site-packages\pandas\core\generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [15]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
Pclass      891 non-null int64
Age         891 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
Sex         891 non-null object
Embarked    891 non-null object
dtypes: float64(2), int64(3), object(2)
memory usage: 48.8+ KB


# 將字串類型的類別資料做轉換

In [16]:
X_train = pd.get_dummies(X_train)

In [17]:
X_train.head()

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,3,22.0,1,0,7.2500,0,1,0,0,1
1,1,38.0,1,0,71.2833,1,0,1,0,0
2,3,26.0,0,0,7.9250,1,0,0,0,1
3,1,35.0,1,0,53.1000,1,0,0,0,1
4,3,35.0,0,0,8.0500,0,1,0,0,1


In [18]:
X_test = pd.get_dummies(X_test)

In [19]:
X_test.head()

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,3,34.5,0,0,7.8292,0,1,0,1,0
1,3,47.0,1,0,7.0000,1,0,0,0,1
2,2,62.0,0,0,9.6875,0,1,0,1,0
3,3,27.0,0,0,8.6625,0,1,0,0,1
4,3,22.0,1,1,12.2875,1,0,0,0,1


# 建立keras模型

In [20]:
model = Sequential()

In [21]:
model = Sequential()  
model.add(Dense(units=40,input_dim = 10,kernel_initializer='uniform',activation='relu'))  
model.add(Dense(30,activation='relu'))  
model.add(Dense(1,activation='sigmoid'))

In [22]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])  

In [23]:
train_his = model.fit(X_train ,y_train ,validation_split=0.1, epochs=100, batch_size=32, verbose=2)

Train on 801 samples, validate on 90 samples
Epoch 1/100
 - 0s - loss: 0.6484 - acc: 0.6542 - val_loss: 0.5530 - val_acc: 0.7222
Epoch 2/100
 - 0s - loss: 0.5942 - acc: 0.7016 - val_loss: 0.5238 - val_acc: 0.7444
Epoch 3/100
 - 0s - loss: 0.5678 - acc: 0.7066 - val_loss: 0.5023 - val_acc: 0.7222
Epoch 4/100
 - 0s - loss: 0.5628 - acc: 0.7228 - val_loss: 0.4775 - val_acc: 0.8000
Epoch 5/100
 - 0s - loss: 0.5348 - acc: 0.7316 - val_loss: 0.4715 - val_acc: 0.7889
Epoch 6/100
 - 0s - loss: 0.5438 - acc: 0.7278 - val_loss: 0.4926 - val_acc: 0.7222
Epoch 7/100
 - 0s - loss: 0.5360 - acc: 0.7541 - val_loss: 0.4733 - val_acc: 0.7889
Epoch 8/100
 - 0s - loss: 0.5187 - acc: 0.7491 - val_loss: 0.4896 - val_acc: 0.7667
Epoch 9/100
 - 0s - loss: 0.5158 - acc: 0.7665 - val_loss: 0.4478 - val_acc: 0.8111
Epoch 10/100
 - 0s - loss: 0.4828 - acc: 0.7803 - val_loss: 0.4216 - val_acc: 0.8111
Epoch 11/100
 - 0s - loss: 0.5290 - acc: 0.7640 - val_loss: 0.4299 - val_acc: 0.8222
Epoch 12/100
 - 0s - loss: 0.

Epoch 97/100
 - 0s - loss: 0.4130 - acc: 0.8215 - val_loss: 0.3622 - val_acc: 0.8556
Epoch 98/100
 - 0s - loss: 0.4114 - acc: 0.8252 - val_loss: 0.3631 - val_acc: 0.8556
Epoch 99/100
 - 0s - loss: 0.4070 - acc: 0.8190 - val_loss: 0.3542 - val_acc: 0.8333
Epoch 100/100
 - 0s - loss: 0.4013 - acc: 0.8290 - val_loss: 0.3464 - val_acc: 0.8444


In [24]:
scores = model.evaluate(X_train, y_train)  

891/891 [==============================] - 0s 26us/step


In [25]:
survived_predict = model.predict(X_test)
survived_predict = np.array(survived_predict)
survived_predict = np.round(survived_predict)

In [26]:
survived_predict

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],

In [29]:
submission=pd.DataFrame({'PassengerId':test['PassengerId'], 'Survived':survived_predict[0:418,0]})

In [30]:
submission.to_csv('submission.csv', index=False)